In [1]:
#!/usr/bin/env python3
"""
Enterprise Module Inspector

A comprehensive tool for analyzing Python modules and packages with advanced features
for enterprise environments including detailed documentation extraction, dependency
analysis, security scanning, and multiple output formats.

Author: Enterprise Development Team
Version: 1.0.0
License: MIT
"""

import inspect
import pkgutil
import importlib
import sys
import ast
import json
import csv
import xml.etree.ElementTree as ET
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple, Set
from dataclasses import dataclass, asdict
from datetime import datetime
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
import argparse
import yaml
from collections import defaultdict
import hashlib
import importlib.util

In [ ]:
@dataclass
class MethodInfo:
    """Detailed information about a method or function."""
    name: str
    type: str  # 'function', 'method', 'classmethod', 'staticmethod', 'property'
    signature: str
    docstring: Optional[str]
    source_file: Optional[str]
    line_number: Optional[int]
    is_async: bool
    is_private: bool
    is_dunder: bool
    parameters: List[str]
    return_annotation: Optional[str]
    decorators: List[str]
    complexity_score: int
    dependencies: List[str]

In [ ]:
@dataclass
class ClassInfo:
    """Detailed information about a class."""
    name: str
    bases: List[str]
    methods: List[MethodInfo]
    properties: List[str]
    docstring: Optional[str]
    source_file: Optional[str]
    line_number: Optional[int]
    is_abstract: bool
    mro: List[str]  # Method Resolution Order
    metaclass: Optional[str]


In [ ]:
@dataclass
class ModuleInfo:
    """Detailed information about a module."""
    name: str
    file_path: Optional[str]
    functions: List[MethodInfo]
    classes: List[ClassInfo]
    constants: Dict[str, Any]
    imports: List[str]
    docstring: Optional[str]
    size_bytes: Optional[int]
    last_modified: Optional[datetime]
    version: Optional[str]
    author: Optional[str]
    license: Optional[str]
    dependencies: Set[str]
    security_issues: List[str]


In [ ]:
class SecurityScanner:
    """Basic security scanner for Python modules."""

    SECURITY_PATTERNS = [
        ('eval', 'Use of eval() can be dangerous'),
        ('exec', 'Use of exec() can be dangerous'),
        ('__import__', 'Dynamic imports can be risky'),
        ('open(', 'File operations - check for path traversal'),
        ('subprocess', 'Subprocess execution - validate inputs'),
        ('os.system', 'Shell command execution - high risk'),
        ('pickle.loads', 'Pickle deserialization can execute arbitrary code'),
    ]

    @staticmethod
    def scan_source(source_code: str) -> List[str]:
        """Scan source code for potential security issues."""
        issues = []
        if not source_code:
            return issues

        for pattern, message in SecurityScanner.SECURITY_PATTERNS:
            if pattern in source_code:
                issues.append(f"{pattern}: {message}")
        return issues


In [ ]:
class EnterpriseModuleInspector:
    """
    Enterprise-grade module inspector with comprehensive analysis capabilities.
    """

    def __init__(self,
                 max_workers: int = 4,
                 include_private: bool = False,
                 include_security_scan: bool = True,
                 cache_enabled: bool = True):
        """
        Initialize the inspector with configuration options.

        Args:
            max_workers: Number of threads for parallel processing
            include_private: Whether to include private methods/classes
            include_security_scan: Whether to perform security scanning
            cache_enabled: Whether to cache results
        """
        self.max_workers = max_workers
        self.include_private = include_private
        self.include_security_scan = include_security_scan
        self.cache_enabled = cache_enabled
        self._cache = {}
        self.logger = self._setup_logger()

    def _setup_logger(self) -> logging.Logger:
        """Setup logging configuration."""
        logger = logging.getLogger('EnterpriseModuleInspector')
        logger.setLevel(logging.INFO)

        if not logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            logger.addHandler(handler)

        return logger

    def _calculate_complexity(self, func_obj: Any) -> int:
        """Calculate cyclomatic complexity of a function."""
        try:
            source = inspect.getsource(func_obj)
            tree = ast.parse(source)

            complexity = 1  # Base complexity
            for node in ast.walk(tree):
                if isinstance(node, (ast.If, ast.While, ast.For, ast.AsyncFor,
                                   ast.ExceptHandler, ast.With, ast.AsyncWith)):
                    complexity += 1
                elif isinstance(node, ast.BoolOp):
                    complexity += len(node.values) - 1

            return complexity
        except:
            return 0

    def _extract_decorators(self, func_obj: Any) -> List[str]:
        """Extract decorator names from a function."""
        try:
            source = inspect.getsource(func_obj)
            tree = ast.parse(source)
            decorators = []

            for node in ast.walk(tree):
                if isinstance(node, ast.FunctionDef):
                    for decorator in node.decorator_list:
                        if isinstance(decorator, ast.Name):
                            decorators.append(decorator.id)
                        elif isinstance(decorator, ast.Attribute):
                            decorators.append(decorator.attr)
                    break

            return decorators
        except:
            return []

    def _analyze_method(self, name: str, obj: Any, source_file: str = None) -> MethodInfo:
        """Analyze a single method or function."""
        try:
            signature = str(inspect.signature(obj))
        except:
            signature = "Unable to determine signature"

        try:
            source_file_path = inspect.getfile(obj)
            line_number = inspect.getsourcelines(obj)[1]
        except:
            source_file_path = source_file
            line_number = None

        return MethodInfo(
            name=name,
            type=self._get_method_type(obj),
            signature=signature,
            docstring=inspect.getdoc(obj),
            source_file=source_file_path,
            line_number=line_number,
            is_async=inspect.iscoroutinefunction(obj),
            is_private=name.startswith('_') and not name.startswith('__'),
            is_dunder=name.startswith('__') and name.endswith('__'),
            parameters=list(inspect.signature(obj).parameters.keys()) if hasattr(obj, '__code__') else [],
            return_annotation=self._get_return_annotation(obj),
            decorators=self._extract_decorators(obj),
            complexity_score=self._calculate_complexity(obj),
            dependencies=self._extract_dependencies(obj)
        )

    def _get_method_type(self, obj: Any) -> str:
        """Determine the type of method/function."""
        if inspect.isfunction(obj):
            return 'function'
        elif inspect.ismethod(obj):
            return 'method'
        elif isinstance(obj, classmethod):
            return 'classmethod'
        elif isinstance(obj, staticmethod):
            return 'staticmethod'
        elif isinstance(obj, property):
            return 'property'
        else:
            return 'unknown'

    def _get_return_annotation(self, obj: Any) -> Optional[str]:
        """Get return type annotation."""
        try:
            sig = inspect.signature(obj)
            return str(sig.return_annotation) if sig.return_annotation != inspect.Signature.empty else None
        except:
            return None

    def _extract_dependencies(self, obj: Any) -> List[str]:
        """Extract dependencies from function/method source."""
        try:
            source = inspect.getsource(obj)
            tree = ast.parse(source)
            deps = set()

            for node in ast.walk(tree):
                if isinstance(node, ast.Import):
                    for alias in node.names:
                        deps.add(alias.name)
                elif isinstance(node, ast.ImportFrom):
                    if node.module:
                        deps.add(node.module)

            return list(deps)
        except:
            return []

    def _analyze_class(self, name: str, cls: type, source_file: str = None) -> ClassInfo:
        """Analyze a single class."""
        methods = []
        properties = []

        for method_name, method_obj in inspect.getmembers(cls):
            if not self.include_private and method_name.startswith('_'):
                continue

            if inspect.isfunction(method_obj) or inspect.ismethod(method_obj):
                methods.append(self._analyze_method(method_name, method_obj, source_file))
            elif isinstance(method_obj, property):
                properties.append(method_name)

        try:
            source_file_path = inspect.getfile(cls)
            line_number = inspect.getsourcelines(cls)[1]
        except:
            source_file_path = source_file
            line_number = None

        return ClassInfo(
            name=name,
            bases=[base.__name__ for base in cls.__bases__],
            methods=methods,
            properties=properties,
            docstring=inspect.getdoc(cls),
            source_file=source_file_path,
            line_number=line_number,
            is_abstract=inspect.isabstract(cls),
            mro=[c.__name__ for c in cls.__mro__],
            metaclass=type(cls).__name__ if type(cls) != type else None
        )

    def _extract_module_metadata(self, module: Any) -> Tuple[str, str, str, str]:
        """Extract metadata from module."""
        version = getattr(module, '__version__', None)
        author = getattr(module, '__author__', None)
        license_info = getattr(module, '__license__', None)

        # Try to get file modification time
        try:
            file_path = inspect.getfile(module)
            size_bytes = Path(file_path).stat().st_size
            last_modified = datetime.fromtimestamp(Path(file_path).stat().st_mtime)
        except:
            size_bytes = None
            last_modified = None

        return version, author, license_info, size_bytes, last_modified

    def _analyze_module(self, module_name: str) -> ModuleInfo:
        """Analyze a single module comprehensively."""
        cache_key = f"{module_name}_{datetime.now().date()}"

        if self.cache_enabled and cache_key in self._cache:
            return self._cache[cache_key]

        try:
            module = importlib.import_module(module_name)

            functions = []
            classes = []
            constants = {}
            imports = set()
            security_issues = []

            # Get file path
            try:
                file_path = inspect.getfile(module)

                # Security scan if enabled
                if self.include_security_scan:
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            source_code = f.read()
                        security_issues = SecurityScanner.scan_source(source_code)
                    except:
                        pass

            except:
                file_path = None

            # Analyze module members
            for name, obj in inspect.getmembers(module):
                if not self.include_private and name.startswith('_'):
                    continue

                if inspect.isfunction(obj) and obj.__module__ == module_name:
                    functions.append(self._analyze_method(name, obj, file_path))
                elif inspect.isclass(obj) and obj.__module__ == module_name:
                    classes.append(self._analyze_class(name, obj, file_path))
                elif not inspect.ismodule(obj) and not callable(obj):
                    constants[name] = repr(obj)[:100]  # Truncate long constants

            # Extract module metadata
            version, author, license_info, size_bytes, last_modified = self._extract_module_metadata(module)

            # Extract dependencies
            dependencies = set()
            for func in functions:
                dependencies.update(func.dependencies)
            for cls in classes:
                for method in cls.methods:
                    dependencies.update(method.dependencies)

            module_info = ModuleInfo(
                name=module_name,
                file_path=file_path,
                functions=functions,
                classes=classes,
                constants=constants,
                imports=list(imports),
                docstring=inspect.getdoc(module),
                size_bytes=size_bytes,
                last_modified=last_modified,
                version=version,
                author=author,
                license=license_info,
                dependencies=dependencies,
                security_issues=security_issues
            )

            if self.cache_enabled:
                self._cache[cache_key] = module_info

            return module_info

        except Exception as e:
            self.logger.error(f"Error analyzing module {module_name}: {e}")
            return ModuleInfo(
                name=module_name,
                file_path=None,
                functions=[],
                classes=[],
                constants={},
                imports=[],
                docstring=f"Error importing module: {e}",
                size_bytes=None,
                last_modified=None,
                version=None,
                author=None,
                license=None,
                dependencies=set(),
                security_issues=[]
            )

    def inspect_package(self, package_name: str) -> Dict[str, ModuleInfo]:
        """
        Comprehensively inspect a package and all its submodules.

        Args:
            package_name: Name of the package to inspect

        Returns:
            Dictionary mapping module names to ModuleInfo objects
        """
        self.logger.info(f"Starting inspection of package: {package_name}")

        try:
            package = importlib.import_module(package_name)
        except ImportError as e:
            self.logger.error(f"Failed to import package {package_name}: {e}")
            return {}

        # Collect all module names
        module_names = [package_name]  # Include the main package

        if hasattr(package, '__path__'):
            for loader, module_name, is_pkg in pkgutil.walk_packages(
                package.__path__, package.__name__ + "."
            ):
                module_names.append(module_name)

        # Analyze modules in parallel
        results = {}

        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            future_to_module = {
                executor.submit(self._analyze_module, module_name): module_name
                for module_name in module_names
            }

            for future in as_completed(future_to_module):
                module_name = future_to_module[future]
                try:
                    results[module_name] = future.result()
                    self.logger.info(f"Completed analysis of {module_name}")
                except Exception as e:
                    self.logger.error(f"Error analyzing {module_name}: {e}")
                    results[module_name] = ModuleInfo(
                        name=module_name,
                        file_path=None,
                        functions=[],
                        classes=[],
                        constants={},
                        imports=[],
                        docstring=f"Analysis failed: {e}",
                        size_bytes=None,
                        last_modified=None,
                        version=None,
                        author=None,
                        license=None,
                        dependencies=set(),
                        security_issues=[]
                    )

        self.logger.info(f"Completed inspection of package: {package_name}")
        return results

    def generate_report(self,
                       results: Dict[str, ModuleInfo],
                       output_format: str = 'json',
                       output_file: Optional[str] = None) -> str:
        """
        Generate a comprehensive report in various formats.

        Args:
            results: Analysis results from inspect_package
            output_format: Format for output ('json', 'yaml', 'xml', 'csv', 'html')
            output_file: Optional file path to save the report

        Returns:
            String representation of the report
        """
        if output_format.lower() == 'json':
            report = self._generate_json_report(results)
        elif output_format.lower() == 'yaml':
            report = self._generate_yaml_report(results)
        elif output_format.lower() == 'xml':
            report = self._generate_xml_report(results)
        elif output_format.lower() == 'csv':
            report = self._generate_csv_report(results)
        elif output_format.lower() == 'html':
            report = self._generate_html_report(results)
        else:
            raise ValueError(f"Unsupported output format: {output_format}")

        if output_file:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(report)
            self.logger.info(f"Report saved to {output_file}")

        return report

    def _generate_json_report(self, results: Dict[str, ModuleInfo]) -> str:
        """Generate JSON report."""
        # Convert dataclasses to dictionaries and handle sets
        json_results = {}
        for module_name, info in results.items():
            json_results[module_name] = asdict(info)
            json_results[module_name]['dependencies'] = list(info.dependencies)
            if info.last_modified:
                json_results[module_name]['last_modified'] = info.last_modified.isoformat()

        return json.dumps(json_results, indent=2, default=str)

    def _generate_yaml_report(self, results: Dict[str, ModuleInfo]) -> str:
        """Generate YAML report."""
        yaml_data = {}
        for module_name, info in results.items():
            data = asdict(info)
            data['dependencies'] = list(info.dependencies)
            if info.last_modified:
                data['last_modified'] = info.last_modified.isoformat()
            yaml_data[module_name] = data

        return yaml.dump(yaml_data, default_flow_style=False)

    def _generate_xml_report(self, results: Dict[str, ModuleInfo]) -> str:
        """Generate XML report."""
        root = ET.Element("PackageInspection")
        root.set("timestamp", datetime.now().isoformat())

        for module_name, info in results.items():
            module_elem = ET.SubElement(root, "Module")
            module_elem.set("name", module_name)

            # Add basic info
            ET.SubElement(module_elem, "file_path").text = info.file_path or ""
            ET.SubElement(module_elem, "docstring").text = info.docstring or ""
            ET.SubElement(module_elem, "version").text = info.version or ""

            # Add functions
            functions_elem = ET.SubElement(module_elem, "Functions")
            for func in info.functions:
                func_elem = ET.SubElement(functions_elem, "Function")
                func_elem.set("name", func.name)
                func_elem.set("type", func.type)
                ET.SubElement(func_elem, "signature").text = func.signature

        return ET.tostring(root, encoding='unicode')

    def _generate_csv_report(self, results: Dict[str, ModuleInfo]) -> str:
        """Generate CSV report with flattened data."""
        import io

        output = io.StringIO()
        writer = csv.writer(output)

        # Write headers
        writer.writerow([
            'Module', 'Item_Type', 'Name', 'Signature', 'File_Path',
            'Line_Number', 'Is_Private', 'Complexity', 'Security_Issues'
        ])

        # Write data
        for module_name, info in results.items():
            # Functions
            for func in info.functions:
                writer.writerow([
                    module_name, 'function', func.name, func.signature,
                    func.source_file, func.line_number, func.is_private,
                    func.complexity_score, '; '.join(info.security_issues)
                ])

            # Classes and their methods
            for cls in info.classes:
                writer.writerow([
                    module_name, 'class', cls.name, '', cls.source_file,
                    cls.line_number, cls.name.startswith('_'), '',
                    '; '.join(info.security_issues)
                ])

                for method in cls.methods:
                    writer.writerow([
                        module_name, f'class_method', f"{cls.name}.{method.name}",
                        method.signature, method.source_file, method.line_number,
                        method.is_private, method.complexity_score, ''
                    ])

        return output.getvalue()

    def _generate_html_report(self, results: Dict[str, ModuleInfo]) -> str:
        """Generate HTML report."""
        html = """
        <!DOCTYPE html>
        <html>
        <head>
            <title>Module Inspection Report</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 20px; }
                .module { border: 1px solid #ccc; margin: 10px 0; padding: 10px; }
                .function, .class { margin-left: 20px; padding: 5px; border-left: 2px solid #eee; }
                .security-issue { color: red; font-weight: bold; }
                .private { color: #666; }
                .complexity-high { color: orange; }
            </style>
        </head>
        <body>
            <h1>Module Inspection Report</h1>
            <p>Generated on: {}</p>
        """.format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

        for module_name, info in results.items():
            html += f'<div class="module"><h2>{module_name}</h2>'

            if info.docstring:
                html += f'<p><strong>Description:</strong> {info.docstring[:200]}...</p>'

            if info.security_issues:
                html += '<div class="security-issue">Security Issues:<ul>'
                for issue in info.security_issues:
                    html += f'<li>{issue}</li>'
                html += '</ul></div>'

            # Functions
            if info.functions:
                html += '<h3>Functions</h3>'
                for func in info.functions:
                    css_class = 'private' if func.is_private else ''
                    if func.complexity_score > 10:
                        css_class += ' complexity-high'

                    html += f'<div class="function {css_class}">'
                    html += f'<strong>{func.name}</strong>{func.signature}'
                    if func.complexity_score > 5:
                        html += f' <span class="complexity-high">(Complexity: {func.complexity_score})</span>'
                    html += '</div>'

            # Classes
            if info.classes:
                html += '<h3>Classes</h3>'
                for cls in info.classes:
                    html += f'<div class="class"><strong>{cls.name}</strong>'
                    if cls.bases:
                        html += f' (inherits from: {", ".join(cls.bases)})'

                    for method in cls.methods[:5]:  # Show first 5 methods
                        css_class = 'private' if method.is_private else ''
                        html += f'<div class="function {css_class}">{method.name}{method.signature}</div>'

                    if len(cls.methods) > 5:
                        html += f'<p>... and {len(cls.methods) - 5} more methods</p>'

                    html += '</div>'

            html += '</div>'

        html += '</body></html>'
        return html

    def get_summary_statistics(self, results: Dict[str, ModuleInfo]) -> Dict[str, Any]:
        """Generate summary statistics from inspection results."""
        stats = {
            'total_modules': len(results),
            'total_functions': 0,
            'total_classes': 0,
            'total_methods': 0,
            'modules_with_security_issues': 0,
            'average_complexity': 0,
            'most_complex_functions': [],
            'largest_modules': [],
            'dependencies': set(),
        }

        complexities = []
        module_sizes = []

        for module_name, info in results.items():
            stats['total_functions'] += len(info.functions)
            stats['total_classes'] += len(info.classes)

            if info.security_issues:
                stats['modules_with_security_issues'] += 1

            if info.size_bytes:
                module_sizes.append((module_name, info.size_bytes))

            stats['dependencies'].update(info.dependencies)

            # Collect function complexities
            for func in info.functions:
                complexities.append((f"{module_name}.{func.name}", func.complexity_score))
                stats['total_methods'] += 1

            # Collect method complexities
            for cls in info.classes:
                for method in cls.methods:
                    complexities.append(f"{module_name}.{cls.name}.{method.name}", method.complexity_score)
                    stats['total_methods'] += 1

        if complexities:
            stats['average_complexity'] = sum(c[1] for c in complexities) / len(complexities)
            stats['most_complex_functions'] = sorted(complexities, key=lambda x: x[1], reverse=True)[:10]

        if module_sizes:
            stats['largest_modules'] = sorted(module_sizes, key=lambda x: x[1], reverse=True)[:10]

        stats['total_dependencies'] = len(stats['dependencies'])
        stats['dependencies'] = list(stats['dependencies'])

        return stats



In [ ]:
def main():
    """Command line interface for the Enterprise Module Inspector."""
    parser = argparse.ArgumentParser(description='Enterprise Module Inspector')
    parser.add_argument('package', help='Package name to inspect')
    parser.add_argument('--output-format', choices=['json', 'yaml', 'xml', 'csv', 'html'],
                       default='json', help='Output format')
    parser.add_argument('--output-file', help='Output file path')
    parser.add_argument('--include-private', action='store_true', help='Include private methods')
    parser.add_argument('--no-security-scan', action='store_true', help='Disable security scanning')
    parser.add_argument('--max-workers', type=int, default=4, help='Number of worker threads')
    parser.add_argument('--summary', action='store_true', help='Show summary statistics')
    parser.add_argument('--verbose', action='store_true', help='Enable verbose logging')

    args = parser.parse_args()

    if args.verbose:
        logging.getLogger('EnterpriseModuleInspector').setLevel(logging.DEBUG)

    # Initialize inspector
    inspector = EnterpriseModuleInspector(
        max_workers=args.max_workers,
        include_private=args.include_private,
        include_security_scan=not args.no_security_scan
    )

    # Perform inspection
    results = inspector.inspect_package(args.package)

    if args.summary:
        stats = inspector.get_summary_statistics(results)
        print("\n=== SUMMARY STATISTICS ===")
        for key, value in stats.items():
            if isinstance(value, list) and len(value) > 3:
                print(f"{key}: {len(value)} items (showing first 3)")
                for item in value[:3]:
                    print(f"  - {item}")
            else:
                print(f"{key}: {value}")
        print()

    # Generate report
    report = inspector.generate_report(results, args.output_format, args.output_file)

    if not args.output_file:
        print(report)



In [ ]:
class ModuleInspectorConfig:
    """Configuration management for the module inspector."""

    def __init__(self, config_file: Optional[str] = None):
        self.config = self._load_default_config()
        if config_file and Path(config_file).exists():
            self._load_config_file(config_file)

    def _load_default_config(self) -> Dict[str, Any]:
        """Load default configuration."""
        return {
            'inspector': {
                'max_workers': 4,
                'include_private': False,
                'include_security_scan': True,
                'cache_enabled': True,
                'max_complexity_threshold': 10,
                'max_line_length': 100
            },
            'security': {
                'custom_patterns': [],
                'severity_levels': ['low', 'medium', 'high'],
                'exclude_patterns': []
            },
            'output': {
                'default_format': 'json',
                'include_source_code': False,
                'max_docstring_length': 500,
                'truncate_constants': True
            },
            'filters': {
                'min_complexity_score': 0,
                'exclude_modules': [],
                'include_only_modules': [],
                'exclude_functions': [],
                'file_size_limit_mb': 100
            }
        }

    def _load_config_file(self, config_file: str):
        """Load configuration from file (YAML or JSON)."""
        with open(config_file, 'r') as f:
            if config_file.endswith('.yaml') or config_file.endswith('.yml'):
                file_config = yaml.safe_load(f)
            else:
                file_config = json.load(f)

        # Deep merge configuration
        self._deep_merge(self.config, file_config)

    def _deep_merge(self, base: dict, update: dict):
        """Deep merge two dictionaries."""
        for key, value in update.items():
            if key in base and isinstance(base[key], dict) and isinstance(value, dict):
                self._deep_merge(base[key], value)
            else:
                base[key] = value

    def get(self, key: str, default=None):
        """Get configuration value using dot notation."""
        keys = key.split('.')
        value = self.config
        for k in keys:
            if isinstance(value, dict) and k in value:
                value = value[k]
            else:
                return default
        return value


In [ ]:
class AdvancedMetrics:
    """Advanced metrics calculator for code analysis."""

    @staticmethod
    def calculate_maintainability_index(complexity: int, lines_of_code: int, halstead_volume: float = 0) -> float:
        """Calculate maintainability index."""
        if lines_of_code == 0:
            return 100

        # Simplified MI calculation (original formula requires Halstead metrics)
        mi = 171 - 5.2 * complexity - 0.23 * lines_of_code - 16.2 * (halstead_volume if halstead_volume > 0 else 1)
        return max(0, min(100, mi))

    @staticmethod
    def calculate_technical_debt_ratio(complexity: int, lines_of_code: int) -> float:
        """Estimate technical debt ratio."""
        if lines_of_code == 0:
            return 0

        # Simple heuristic: high complexity relative to code size indicates debt
        debt_factor = (complexity - 1) / lines_of_code if lines_of_code > 0 else 0
        return min(1.0, debt_factor * 10)  # Cap at 100%

    @staticmethod
    def calculate_lines_of_code(source: str) -> Tuple[int, int, int]:
        """Calculate lines of code metrics (total, code, comments)."""
        if not source:
            return 0, 0, 0

        lines = source.split('\n')
        total_lines = len(lines)
        code_lines = 0
        comment_lines = 0

        for line in lines:
            stripped = line.strip()
            if not stripped:
                continue
            elif stripped.startswith('#'):
                comment_lines += 1
            else:
                code_lines += 1
                if '#' in line:  # Inline comment
                    comment_lines += 1

        return total_lines, code_lines, comment_lines

In [ ]:
class CodeQualityAnalyzer:
    """Analyze code quality patterns and anti-patterns."""

    QUALITY_PATTERNS = {
        'god_function': {
            'pattern': lambda complexity, loc: complexity > 15 and loc > 100,
            'message': 'Function is too complex and long (God Function anti-pattern)',
            'severity': 'high'
        },
        'deep_nesting': {
            'pattern': lambda complexity, loc: complexity > 10,
            'message': 'Function has deep nesting levels',
            'severity': 'medium'
        },
        'long_parameter_list': {
            'pattern': lambda params: len(params) > 7,
            'message': 'Function has too many parameters',
            'severity': 'medium'
        },
        'unclear_naming': {
            'pattern': lambda name: len(name) < 3 or name in ['func', 'method', 'data', 'obj'],
            'message': 'Function name is unclear or too generic',
            'severity': 'low'
        }
    }

    @classmethod
    def analyze_function_quality(cls, method_info: MethodInfo, source_code: str = '') -> List[Dict[str, str]]:
        """Analyze function quality and return issues found."""
        issues = []

        if source_code:
            total_lines, code_lines, _ = AdvancedMetrics.calculate_lines_of_code(source_code)
        else:
            total_lines = code_lines = 0

        # Check god function
        if cls.QUALITY_PATTERNS['god_function']['pattern'](method_info.complexity_score, code_lines):
            issues.append({
                'type': 'god_function',
                'message': cls.QUALITY_PATTERNS['god_function']['message'],
                'severity': cls.QUALITY_PATTERNS['god_function']['severity']
            })

        # Check deep nesting
        if cls.QUALITY_PATTERNS['deep_nesting']['pattern'](method_info.complexity_score, code_lines):
            issues.append({
                'type': 'deep_nesting',
                'message': cls.QUALITY_PATTERNS['deep_nesting']['message'],
                'severity': cls.QUALITY_PATTERNS['deep_nesting']['severity']
            })

        # Check parameter list length
        if cls.QUALITY_PATTERNS['long_parameter_list']['pattern'](method_info.parameters):
            issues.append({
                'type': 'long_parameter_list',
                'message': cls.QUALITY_PATTERNS['long_parameter_list']['message'],
                'severity': cls.QUALITY_PATTERNS['long_parameter_list']['severity']
            })

        # Check naming
        if cls.QUALITY_PATTERNS['unclear_naming']['pattern'](method_info.name):
            issues.append({
                'type': 'unclear_naming',
                'message': cls.QUALITY_PATTERNS['unclear_naming']['message'],
                'severity': cls.QUALITY_PATTERNS['unclear_naming']['severity']
            })

        return issues

In [ ]:
class ModuleComparator:
    """Compare different versions or instances of modules."""

    @staticmethod
    def compare_modules(old_info: ModuleInfo, new_info: ModuleInfo) -> Dict[str, Any]:
        """Compare two module versions and return differences."""
        comparison = {
            'name': new_info.name,
            'changes': {
                'added_functions': [],
                'removed_functions': [],
                'modified_functions': [],
                'added_classes': [],
                'removed_classes': [],
                'modified_classes': [],
                'complexity_changes': {},
                'size_change': None,
                'dependency_changes': {
                    'added': [],
                    'removed': []
                }
            }
        }

        # Function comparison
        old_functions = {f.name: f for f in old_info.functions}
        new_functions = {f.name: f for f in new_info.functions}

        comparison['changes']['added_functions'] = list(set(new_functions.keys()) - set(old_functions.keys()))
        comparison['changes']['removed_functions'] = list(set(old_functions.keys()) - set(new_functions.keys()))

        for func_name in set(old_functions.keys()) & set(new_functions.keys()):
            old_func = old_functions[func_name]
            new_func = new_functions[func_name]

            if (old_func.signature != new_func.signature or
                old_func.complexity_score != new_func.complexity_score):
                comparison['changes']['modified_functions'].append({
                    'name': func_name,
                    'old_complexity': old_func.complexity_score,
                    'new_complexity': new_func.complexity_score,
                    'signature_changed': old_func.signature != new_func.signature
                })

        # Class comparison (similar logic)
        old_classes = {c.name: c for c in old_info.classes}
        new_classes = {c.name: c for c in new_info.classes}

        comparison['changes']['added_classes'] = list(set(new_classes.keys()) - set(old_classes.keys()))
        comparison['changes']['removed_classes'] = list(set(old_classes.keys()) - set(new_classes.keys()))

        # Size comparison
        if old_info.size_bytes and new_info.size_bytes:
            comparison['changes']['size_change'] = new_info.size_bytes - old_info.size_bytes

        # Dependency comparison
        old_deps = set(old_info.dependencies)
        new_deps = set(new_info.dependencies)
        comparison['changes']['dependency_changes']['added'] = list(new_deps - old_deps)
        comparison['changes']['dependency_changes']['removed'] = list(old_deps - new_deps)

        return comparison

In [ ]:
class ReportGenerator:
    """Enhanced report generator with multiple output formats and templates."""

    def __init__(self, config: ModuleInspectorConfig):
        self.config = config

    def generate_executive_summary(self, results: Dict[str, ModuleInfo], stats: Dict[str, Any]) -> str:
        """Generate executive summary report."""
        summary = f"""
# Executive Summary - Code Analysis Report

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Overview
- **Total Modules Analyzed:** {stats['total_modules']}
- **Total Functions:** {stats['total_functions']}
- **Total Classes:** {stats['total_classes']}
- **Total Methods:** {stats['total_methods']}

## Key Findings

### Code Quality
- **Average Complexity Score:** {stats.get('average_complexity', 0):.2f}
- **Modules with Security Issues:** {stats['modules_with_security_issues']}
- **High Complexity Functions:** {len([f for f in stats.get('most_complex_functions', []) if f[1] > 15])}

### Dependencies
- **Total External Dependencies:** {stats['total_dependencies']}
- **Most Common Dependencies:** {', '.join(list(stats['dependencies'])[:5])}

### Recommendations
"""

        # Add recommendations based on findings
        if stats['modules_with_security_issues'] > 0:
            summary += f"- **Security Review Required:** {stats['modules_with_security_issues']} modules have potential security issues\n"

        if stats.get('average_complexity', 0) > 10:
            summary += f"- **Refactoring Needed:** Average complexity ({stats.get('average_complexity', 0):.1f}) exceeds recommended threshold\n"

        if stats['total_dependencies'] > 50:
            summary += f"- **Dependency Management:** Consider reducing dependencies ({stats['total_dependencies']} total)\n"

        return summary

    def generate_technical_report(self, results: Dict[str, ModuleInfo]) -> str:
        """Generate detailed technical report."""
        report = f"""
# Technical Analysis Report

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Module Details

"""

        for module_name, info in results.items():
            report += f"### {module_name}\n\n"

            if info.docstring:
                report += f"**Description:** {info.docstring[:200]}...\n\n"

            report += f"- **File Path:** `{info.file_path}`\n"
            report += f"- **Size:** {info.size_bytes} bytes\n" if info.size_bytes else ""
            report += f"- **Functions:** {len(info.functions)}\n"
            report += f"- **Classes:** {len(info.classes)}\n"

            if info.security_issues:
                report += f"- **Security Issues:** {len(info.security_issues)}\n"
                for issue in info.security_issues[:3]:
                    report += f"  - {issue}\n"

            report += "\n"

        return report

In [ ]:
# Enhanced main function with new features
def main():
    """Enhanced command line interface with advanced features."""
    parser = argparse.ArgumentParser(description='Enterprise Module Inspector v2.0')
    parser.add_argument('package', help='Package name to inspect')

    # Output options
    parser.add_argument('--output-format', choices=['json', 'yaml', 'xml', 'csv', 'html'],
                       default='json', help='Output format')
    parser.add_argument('--output-file', help='Output file path')
    parser.add_argument('--config', help='Configuration file (YAML or JSON)')

    # Analysis options
    parser.add_argument('--include-private', action='store_true', help='Include private methods')
    parser.add_argument('--no-security-scan', action='store_true', help='Disable security scanning')
    parser.add_argument('--quality-analysis', action='store_true', help='Enable quality analysis')
    parser.add_argument('--max-workers', type=int, default=4, help='Number of worker threads')

    # Report options
    parser.add_argument('--summary', action='store_true', help='Show summary statistics')
    parser.add_argument('--executive-summary', action='store_true', help='Generate executive summary')
    parser.add_argument('--technical-report', action='store_true', help='Generate technical report')
    parser.add_argument('--compare-with', help='Compare with previous analysis (JSON file)')

    # Filtering options
    parser.add_argument('--min-complexity', type=int, default=0, help='Minimum complexity threshold')
    parser.add_argument('--exclude-modules', nargs='*', help='Modules to exclude')
    parser.add_argument('--include-source', action='store_true', help='Include source code in output')

    # Other options
    parser.add_argument('--verbose', action='store_true', help='Enable verbose logging')
    parser.add_argument('--cache-dir', help='Cache directory path')

    args = parser.parse_args()

    # Setup logging
    if args.verbose:
        logging.getLogger('EnterpriseModuleInspector').setLevel(logging.DEBUG)

    # Load configuration
    config = ModuleInspectorConfig(args.config)

    # Override config with command line arguments
    if args.include_private:
        config.config['inspector']['include_private'] = True
    if args.no_security_scan:
        config.config['inspector']['include_security_scan'] = False
    if args.max_workers:
        config.config['inspector']['max_workers'] = args.max_workers

    # Initialize inspector with config
    inspector = EnterpriseModuleInspector(
        max_workers=config.get('inspector.max_workers', 4),
        include_private=config.get('inspector.include_private', False),
        include_security_scan=config.get('inspector.include_security_scan', True)
    )

    # Perform inspection
    print(f"Analyzing package: {args.package}")
    results = inspector.inspect_package(args.package)

    # Apply filters
    if args.exclude_modules:
        for module in args.exclude_modules:
            results.pop(module, None)

    if args.min_complexity > 0:
        # Filter functions by complexity
        for module_info in results.values():
            module_info.functions = [f for f in module_info.functions if f.complexity_score >= args.min_complexity]

    # Quality analysis
    if args.quality_analysis:
        print("\nPerforming quality analysis...")
        for module_name, info in results.items():
            for func in info.functions:
                try:
                    source_code = inspect.getsource(eval(f"{module_name}.{func.name}")) if hasattr(eval(module_name), func.name) else ""
                    quality_issues = CodeQualityAnalyzer.analyze_function_quality(func, source_code)
                    if quality_issues:
                        print(f"Quality issues in {module_name}.{func.name}:")
                        for issue in quality_issues:
                            print(f"  - {issue['message']} (Severity: {issue['severity']})")
                except:
                    pass

    # Generate reports
    if args.summary or args.executive_summary or args.technical_report:
        stats = inspector.get_summary_statistics(results)

    if args.summary:
        print("\n=== SUMMARY STATISTICS ===")
        for key, value in stats.items():
            if isinstance(value, list) and len(value) > 3:
                print(f"{key}: {len(value)} items (showing first 3)")
                for item in value[:3]:
                    print(f"  - {item}")
            else:
                print(f"{key}: {value}")

    if args.executive_summary:
        report_gen = ReportGenerator(config)
        exec_summary = report_gen.generate_executive_summary(results, stats)
        print(exec_summary)

        if args.output_file:
            summary_file = args.output_file.replace('.', '_executive_summary.')
            with open(summary_file, 'w') as f:
                f.write(exec_summary)

    if args.technical_report:
        report_gen = ReportGenerator(config)
        tech_report = report_gen.generate_technical_report(results)
        print(tech_report)

        if args.output_file:
            tech_file = args.output_file.replace('.', '_technical.')
            with open(tech_file, 'w') as f:
                f.write(tech_report)

    # Module comparison
    if args.compare_with:
        try:
            with open(args.compare_with, 'r') as f:
                old_results_data = json.load(f)

            print("\n=== MODULE COMPARISON ===")
            # Note: This would need proper deserialization logic
            print("Comparison feature requires previous analysis results...")

        except Exception as e:
            print(f"Error loading comparison file: {e}")

    # Generate main report
    report = inspector.generate_report(results, args.output_format, args.output_file)

    if not args.output_file:
        print("\n=== ANALYSIS REPORT ===")
        print(report[:2000] + "..." if len(report) > 2000 else report)

    print(f"\nAnalysis complete! Processed {len(results)} modules.")


if __name__ == '__main__':
    main()